In [1]:
# _____TF-IDF libraries_____
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# _____helper Libraries_____
import pandas as pd
import pickle
import csv
import json
import timeit
import random
import re
import spacy
from tqdm import tqdm

In [2]:
nlp = spacy.load('en_core_web_sm')

In [23]:
csv_dataset = pd.read_csv("dialogueText.csv")
dataset = csv_dataset.iloc[400:100000,1:].values

In [24]:
prev = []
my_list = []
q_a = {'question':'answer'}
question = []
answer = []
q = ""
a = ""
previd=dataset[0][0]
start=0
i = int(0)

row_count = dataset.shape[0]

In [25]:
for x in tqdm(dataset):
    if i < row_count and previd != dataset[i][0]:
        for j in range(start,i):
            from1 = dataset[j][2]
            if (j==start) and dataset[j][3] == dataset[j][3]:
                break
            while (j < row_count) and (dataset[j][3] != dataset[j][3] or dataset[j][3] != from1):
                q = q + str(dataset[j][4])
                j = j+1

            while j < row_count and dataset[j][3] == from1 :#tofrom
                if a == "":
                    a = str(dataset[j][4])
                else:
                    a = a + "\n" + str(dataset[j][4])
                j=j+1
                
            if(a != ""):
                answer.append(a)

            if(a != "" and q != ""):
                question.append(q)
            a=""
            q=""
            if j >= i:
                break
        previd = dataset[i][0]
        start = i
    i=i+1

100%|██████████████████████████████████████████████████████████████████████████| 9600/9600 [00:00<00:00, 137185.51it/s]


In [26]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    return text

In [27]:
def preprocess(userQuery):
    ques = []
    temp = clean_text(userQuery.lower())
    for token in temp.split(' '):
        t = str(token)
        if nlp.vocab[t].is_stop:
            pass
    #         print('no')
        else:
            ques.append(t)
    ques_list = ' '.join(ques)

    ques2 = []
    temp2 = nlp(ques_list)
    for token2 in temp2:
        
        ques2.append(token2.lemma_)
    ques_list = ' '.join(ques2)
    return ques_list

In [28]:
def talk_to_cb_primary(test_set_sentence, minimum_score , json_file_path , tfidf_vectorizer_pikle_path ,tfidf_matrix_train_pikle_path):
    test_set = (test_set_sentence, "")

    try:
        f = open(tfidf_vectorizer_pikle_path, 'rb')
        tfidf_vectorizer = pickle.load(f)
        f.close()

        f = open(tfidf_matrix_train_pikle_path, 'rb')
        tfidf_matrix_train = pickle.load(f)
        f.close()
    except:
        print("NO TRAINING")
        
    tfidf_matrix_test = tfidf_vectorizer.transform(test_set)

    cosine = cosine_similarity(tfidf_matrix_test, tfidf_matrix_train)

    cosine = np.delete(cosine, 0)
    max = cosine.max()
    response_index = 0
    if (max > minimum_score):
        new_max = max - 0.01
        list = np.where(cosine > new_max)
        response_index = random.choice(list[0])
    else :
        return "Apologies, I can't understand. My developers haven't trained me much! :(" , 0
           

    j = 0

    with open(json_file_path, "r") as sentences_file:
        reader = json.load(sentences_file)
        for row in reader:
            j += 1  # we begin with 1 not 0 &    j is initialized by 0
            if j == response_index:
                return row["response"], max
                break

In [9]:
# def previous_chats(query):
#     minimum_score = 0.7
#     file = "data.json"
#     tfidf_vectorizer_pikle_path = "previous_tfidf_vectorizer.pickle"
#     tfidf_matrix_train_path = "previous_tfidf_matrix_train.pickle"
#     query_response, score = talk_to_cb_primary(query , minimum_score , file , tfidf_vectorizer_pikle_path , tfidf_matrix_train_path)
#     return query_response

In [29]:
import string 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
# from nltk.corpus import stopwords
# nltk.download('stopwords')
# stopwords = stopwords.words('english')


In [30]:
def clean_string(text):
    text = ''.join([word for word in text if word not in string.punctuation])
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stopwords])
    
    return text

In [31]:
# cleaned = list(map(clean_string,sentences))
# vectorizer = CountVectorizer().fit_transform(cleaned)
# vectors = vectorizer.toarray()

In [32]:
# csim = cosine_similarity(vectors)

In [33]:
def preprocess(userQuery):
    ques = ""
    temp = clean_text(userQuery.lower())
    for token in temp.split(' '):
        t = str(token)
        if nlp.vocab[t].is_stop:
            pass
    #         print('no')
        else:
            if ques == "":
                ques = t
            else:
                ques = ques + " " + t
    #ques_list = ' '.join(ques)

#     ques2 = []
#     temp2 = nlp(ques_list)
#     for token2 in temp2:
        
#         ques2.append(token2.lemma_)
#     ques_list = ' '.join(ques2)
    return ques

In [34]:
def cosine_sim_vectors(vec1,vec2):
    vec1 = vec1.reshape(1,-1)
    vec2 = vec2.reshape(1,-1)
    
    return cosine_similarity(vec1,vec2)[0][0]

In [45]:
from sklearn.metrics import precision_score

def previous_chats(query):
    minimum_score = 0.6
    file = "data\data\data.json"
    tfidf_vectorizer_pikle_path = "data\data\previous_tfidf_vectorizer.pickle"
    tfidf_matrix_train_path = "data\data\previous_tfidf_matrix_train.pickle"
    query_response, score = talk_to_cb_primary(query , minimum_score , file , tfidf_vectorizer_pikle_path , tfidf_matrix_train_path)
    
    return query_response


In [46]:
y_pred = []
y_true = []
y_true = answer


i = int(0)
precision = []
sentences = []
similarity = []
cleaned = []
predv = ""
flag = 1
for x in question:
    predv = previous_chats(x)
    #print(predv)
    y_pred.append(predv)
    sentences.append(y_pred[i])
    sentences.append(y_true[i])
    txt1 = preprocess(y_pred[i])
    txt2 = preprocess(y_true[i])
    i = i+1
#     print(txt1)
#     print(txt2)
    #print(len(txt1))
    if len(txt1) == 0 or len(txt2) == 0:
        flag = 0
    if flag == 1:
        if len(txt1) != 0 and len(txt2) != 0:
            cleaned.append(txt1)
            cleaned.append(txt2)
# doc_term_matrix = sparse_matrix.todense()
# df = pd.DataFrame(doc_term_matrix, 
#                   columns=count_vectorizer.get_feature_names(), 
#                   index=['y_pred', 'y_true', 'doc_putin'])        
#     print(cleaned)
#     for y in sentences:
#         txt = preprocess(y)
#         print(txt)
#         if txt != txt:
#             flag  = 0
#             break
#         else:
#             cleaned.append(txt)
    if flag == 1:
        vectorizer = CountVectorizer().fit_transform(cleaned)#for getting the count of words present in string
        vectors = vectorizer.toarray()
#         print(txt1)
#         print(txt2)
#         print(vectors)
        similarity.append(cosine_sim_vectors(vectors[0],vectors[1]))
        sentences = []
        cleaned =[]
        prevdv = ""
    flag = 1

apologies, understand. developers haven't trained much! :(
ndiswrapper?
[[1 1 1 1 0 1 1]
 [0 0 0 0 1 0 0]]
linuxjones check xf86config-4
linuxjones check xf86config-4
[[1 1 1]
 [1 1 1]]
isnt dpkg reconfigure thing?
there, thanks
[[1 1 1 0 0 1]
 [0 0 0 1 1 0]]
try --> /join #hsa
try --> /join #hsa
[[1 1 1]
 [1 1 1]]
user preference. use unity+xmonad
user preference. use unity+xmonad
[[1 1 1 1 1]
 [1 1 1 1 1]]
apologies, understand. developers haven't trained much! :(
hello
[[1 1 1 0 1 1 1]
 [0 0 0 1 0 0 0]]
sudo apt-get install command-not-found
yes
sudo apt-get install command-not-found
yes
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
1.0 isn't warty
i think set repo somewhere, check wiki
1.0 isn't warty
i think set repo somewhere, check wiki
[[1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1]]
apologies, understand. developers haven't trained much! :(
greg kroah-hartman talk it, video somewhere
1 hour so, goes examples
[[1 1 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0]
 [0 0 1 1 1 1 0 1 1 1 0 1 1 1 0 0 1]]
think peopl

apologies, understand. developers haven't trained much! :(
nope got options grub all!
 got memtest memtest serial
[[0 1 1 0 0 1 0 1 0 0 0 1 1]
 [1 0 0 2 1 0 2 0 1 1 1 0 0]]
apologies, understand. developers haven't trained much! :(
wines apps trouble? some?
:d
[[1 0 1 1 1 0 1 0 1 0]
 [0 1 0 0 0 1 0 1 0 1]]
apologies, understand. developers haven't trained much! :(
theres tool called 'epidermis' (not repos) install/set themes fro stuff..
 ways change it.. dont worry muich silly splash stuff
[[1 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
 [0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1 2 1 1 1 0 0 1 1]]
apologies, understand. developers haven't trained much! :(
grep lm /proc/cpuinfo
[[1 0 1 0 1 0 1 0 1 1]
 [0 1 0 1 0 1 0 1 0 0]]
use virtual graphics card generic
use virtual graphics card generic
[[1 1 1 1 1]
 [1 1 1 1 1]]
apologies, understand. developers haven't trained much! :(
sure mean
[[1 1 1 0 1 0 1 1]
 [0 0 0 1 0 1 0 0]]
like campcaster software
u click tool user groups, u users right?
[

sudo /etc/init.d/ifupdown restart
sudo /etc/init.d/ifupdown restart
[[1 1 1 1 1]
 [1 1 1 1 1]]
downloaded night, installation hard?
where it!? omg download warty reason
downloaded night, installation hard?
where it!? omg download warty reason
[[1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1]]
apologies, understand. developers haven't trained much! :(
maybe iso corrupt? maybe burned disc fast?
[[1 0 0 1 0 0 1 0 0 1 1 1]
 [0 1 1 0 1 1 0 1 2 0 0 0]]
apologies, understand. developers haven't trained much! :(
remove add back? ubuntu update?
[[0 1 0 1 1 1 0 1 0 1 0]
 [1 0 1 0 0 0 1 0 1 0 1]]
apologies, understand. developers haven't trained much! :(
yeah, success..
[[1 1 1 1 0 1 1 0]
 [0 0 0 0 1 0 0 1]]
apologies, understand. developers haven't trained much! :(
try ctrl enter, shift enter?
[[1 0 1 0 1 1 0 1 0 1]
 [0 1 0 2 0 0 1 0 1 0]]
getdeb.net
getdeb.net
[[1 1]
 [1 1]]
english?
english?
[[1]
 [1]]
try ifconfig <dev name> <ip address>
try ifconfig <dev name> <ip address>
[[1 1 1 1 1 1]
 [1 1 1

isnt problems
think it's switch hal hardware stuff intead udev
[[0 0 0 1 0 1 0 0 0 0]
 [1 1 1 0 1 0 1 1 1 1]]
return 'file suported.'
return 'file suported.'
[[1 1 1]
 [1 1 1]]
yes apt-get install libc6
yes apt-get install libc6
[[1 1 1 1 1]
 [1 1 1 1 1]]
check teh server forums.  real time like irc, good resource nonetheless
check teh server forums.  real time like irc, good resource nonetheless
[[1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]]
apologies, understand. developers haven't trained much! :(
apt-get install linux-source-2.6.10
[[0 1 0 1 0 1 0 0 1 0 1 1]
 [1 0 1 0 1 0 1 1 0 1 0 0]]
yes, wont rights
yes, wont rights
[[1 1 1]
 [1 1 1]]
apologies, understand. developers haven't trained much! :(
expected that. curious physically syncing data.
[[1 0 0 1 0 1 1 0 0 0 1 1]
 [0 1 1 0 1 0 0 1 1 1 0 0]]
worked
worked
[[1]
 [1]]
nice.
theres inticator applets taht let gnome-menu unity :)
 part? :)
[[0 0 0 0 0 1 0 0 0 0]
 [1 1 1 1 1 0 1 1 1 1]]
yes. probably want use xubuntu fluxbuntu, 

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [53]:
def precision_score(y_pred, y_true, trerw):
    l = 0
    for x in similarity:
        if x > 0.3:
            l = l + 1
    string = 'PRECISION: ' + str(l/len(similarity))     
    return string 

In [54]:
threshold = 0

In [55]:
precision_value = precision_score(y_pred, y_true, threshold)
print(precision_value)

PRECISION: 0.4956896551724138


0.3560606060606061

In [20]:
sentences = ['LinuxJones check XF86Config-4','LinuxJones check XF86Config-4']

In [82]:
cleaned = list(map(clean_string,sentences))
vectorizer = CountVectorizer().fit_transform(cleaned)
vectors = vectorizer.toarray()

In [83]:
cosine_sim_vectors(vectors[0],vectors[1])

1.0000000000000002

In [84]:
sentences

['LinuxJones check XF86Config-4', 'LinuxJones check XF86Config-4']